# Objektorientiere Programmierung: Vertiefung
Diese Notebook vertieft einige Konzepte der objektorientierten Programmierung, insbesondere in Hinblick auf Python.

## Geschützte Variablen und Methoden (Kapselung)
### Geschützte Variablen und Methoden
Wir haben gelernt, dass einer der wesentlichen Vorteile von Objektorientierung die Datenkapselung ist. Damit ist gemeint, dass der Zugriff auf Eigenschaften und Methoden eingeschränkt werden kann. 

Manche Programmiersprachen wie z.B. Java markieren diese Zugriffsrechte explizit und sind in der Auslegung sehr strikt. Die folgende Variablendeklaration in Java beschränkt den Zugriff auf die Variable ``score`` auf die Klasse selbst, weil sie die Sichbarkeit der Variable auf ``private`` setzt.

~~~
private int score = 0;
~~~

Dadurch kann der Wert von `score` nur aus der Klasse heraus gelesen oder verändert werden.


Der folgende Code hingegen erlaubt den uneingeschränkten Zugriff auf die Eigenschaft `username`:

~~~
public String username;
~~~

Einen ähnlichen Mechanismus gibt es auch in Python. Allerdings geht man hier die Dinge relaxter an: Ein vor einen Variablennamen oder einen Methodennamen gesetztes Underline (``_``) bedeutet, dass dieser Teil des Objekt von außerhalb des Objekts nicht verwendet, vor allem nicht verändert werden soll.

In [ ]:
class MyClass:
    
    def __init__(self, val):
        self.set_val(val)
        
    def get_val(self):
        return self._val
        
    def set_val(self, val):
        if val > 0:
            self._val = val
        else:
            raise ValueError('val must be greater 0')
        
myclass = MyClass(27)        
myclass._val

Wie wir sehen, ist die Eigenschaft `_val` von außerhalb verfügbar. Allerdings signalisiert das Underline, dass vom Programmierer der Klasse nicht vorgesehen ist, dass dieser Wert direkt verwendet wird (sondern z.B. nur über die Methoden `get_val()` und `set_val()`). Wenn ein anderer Programmierer der Meinung ist, dass er direkten Zugriff auf die Eigenschaft `_val` braucht, liegt das in seiner Verantwortung (wird aber von Python nicht unterbunden). Man spricht hier von *protection by convention*. Python-Programmierer halten sich in aller Regel an diese Konvention, weshalb dieser Art von "Schutz" weit verbreitet ist. 

Ein Vorteil dieser Herangehensweise liegt z.B. darin, dass solche Elemente einfacher getestet werden können, weil ein Test direkt zugreifen kann; ob das den Nachteil aufwiegt, dass das Element nur per Konvention und nicht von der Sprache selbst geschützt ist, muss anhand der Erfordernisse eines Projekt beurteilt werden.

### Unsichtbare Eigenschaften und Methoden
Für paranoide Programmierer bietet Python die Möglichkeit, den Zugriff von außerhalb des Objekt komplett zu unterbinden, indem man statt eines Unterstrichts zwei Unterstriche vor den Namen setzt.

In [ ]:
class MyClass:
    
    def __init__(self, val):
        self.__val = val
        
myclass = MyClass(42)        
myclass.__val

Hier sehen wir, dass die Eigenschaft `__val` von außerhalb der Klasse gar nicht sichtbar und damit auch nicht veränderbar ist. Innerhalb der Klasse ist sie jedoch normal verfügbar:

In [ ]:
class MyClass:
    
    def __init__(self, val):
        self.__val = val
        
    def get_val(self):
        return self.__val
        
myclass = MyClass(42)        
myclass.get_val()

### Datenkapelung mit Properties
Wie wir gesehen haben, werden für den Zugriff auf geschützte Eigenschaften eigene Getter- und Setter-Methoden geschrieben, über die der Wert einer Eigenschaft kontrolliert verändert werden kann. Programmieren wir eine `Student`-Klasse, in der eine Note gespeichert werden soll. Um den Zugriff auf diese Eigenschaft zu kontrollieren, schreiben wir eine Setter- und eine Getter-Methode.

In [ ]:
class GradingError(Exception): pass


class Student:
    
    def __init__(self, matrikelnr):
        self.matrikelnr = matrikelnr
        self._grade = 0
        
    def set_grade(self, grade):
        if grade > 0 and grade < 6:
            self._grade = grade
        else:
            raise ValueError('Grade must be between 1 and 5!')
            
    def get_grade(self):
        if self._grade > 0:
            return self._grade
        raise GradingError('Noch nicht benotet!')

Wir können jetzt die Note setzen und auslesen:

In [ ]:
anna = Student('01754645')
anna.set_grade(6)

In [ ]:
anna.set_grade(2)
anna.get_grade()

Allerdings ist der direkte Zugriff auf `grade` immer noch möglich:

In [ ]:
anna._grade

In [ ]:
anna._grade = 6

Wie wir bereits gesehen haben, können wir das verhindern, indem wir die Eigenschaft `grade` auf `__grade` umbenennen. 

## Properties setzen via Getter und Setter 
Python bietet eine Möglichkeit, das Setzen und Auslesen von Objekteigenschaften automatisch durch Methoden zu leiten. Dazu werden der Getter und Setter an die `poperty`-Funktion übergeben (letzte Zeile der Klasse).

In [ ]:
class Student:
    
    def __init__(self, matrikelnr):
        self.matrikelnr = matrikelnr
        self.__grade = 0
        
    def set_grade(self, grade):
        if grade > 0 and grade < 6:
            self.__grade = grade
        else:
            raise ValueError('Grade must be between 1 and 5!')
            
    def get_grade(self):
        if self.__grade > 0:
            return self.__grade
        raise GradingError('Noch nicht benotet!')
        
    grade = property(get_grade, set_grade)
    
otto = Student('01745646465')    
otto.grade = 6

Wie wir sehen, können wir die Eigenschaft des Objekts direkt setzen und auslesen, der Zugriff wird aber von Python jeweils durch den Setter und Getter geleitet.

Wenn wir nur eine Methode (den Getter) als Argument an die property()-Funktion übergeben, haben wir eine Eigenschaft, die sich nur auslesen, aber nicht verändern lässt.

In [ ]:
class Student:
    
    def __init__(self, matrikelnr, grade):
        self.matrikelnr = matrikelnr
        self.__grade = grade
                
    def get_grade(self):
        if self.__grade > 0:
            return self.__grade
        raise GradingError('Noch nicht benotet!')
        
    grade = property(get_grade)
    
albert = Student('0157897846546', 5)    
albert.grade

Wir können also auf unsere via property() definierte Eigenschaften zugreifen. Wir können `grade` aber nicht verwenden,
um die Eigenschaft zu verändern:

In [ ]:
albert.grade = 1

### Der @property-Dekorator
Dekoratoren erweitern  dynamisch die Funktionalität von Funktionen indem sie diese (im Hintergrund) in eine weitere Funktion verpacken. Die Anwendung eines Dekorators ist einfach: man schreibt ihn einfach vor die Funktionsdefinition.
Python bringt eine Reihe von Dekoratoren mit, man kann sich aber auch eigene Dekoratoren schreiben, was jedoch hier nicht behandelt wird.
Der in Python-Objekten eingebaute `@property`-Dekorator ist eine Alternative zu der oben vorgestellten `property()`-Funktion:

In [ ]:
class Student:
    
    def __init__(self, matrikelnr):
        self.matrikelnr = matrikelnr
        self.__grade = 0
            
    @property
    def grade(self):
        if self.__grade > 0:
            return self.__grade
        raise GradingError('Noch nicht benotet!')
        
    @grade.setter
    def grade(self, grade):
        if grade > 0 and grade < 6:
            self.__grade = grade
        else:
            raise ValueError('Grade must be between 1 and 5!')
    

hugo = Student('0176464645454')    

In [ ]:
hugo.grade = 6

In [ ]:
hugo.grade = 2

In [ ]:
hugo.grade

## Klassenvariablen (Static members)
Wir haben gelernt, dass Klassen Eigenschaften und Methoden von Objekten festlegen. Allerdings (und das kann zu Beginn etwas verwirrend sein), sind Klassen selbst auch Objekte, die Eigenschaften und Methoden haben. Hier ein Beispiel:

In [ ]:
class MyClass:
    
    the_answer = 42
    
    def __init__(self, val):
        self.the_answer = val
        
MyClass.the_answer        

In [ ]:
mc = MyClass(17)
print('Objekteigenschaft:', mc.the_answer)
print('Klasseneigenschaft:', MyClass.the_answer)

Die eine Eigenschaft hängt also am Klassenobjekt, die andere am aus der Klasse erzeugten Objekt. Solche Klassenobjekte können nützlich sein, weil sie in allen aus der Klasse erzeugten Objekten verfügbar sind (sogar via `self`, solange das Objekt nicht selbst eine gleichnamige Eigenschaft hat):

In [ ]:
class MyClass:
    instance_counter = 0
    
    def __init__(self):
        MyClass.instance_counter += 1
        print(f'Ich bin das {MyClass.instance_counter}. Objekt')
        
a = MyClass()
b = MyClass()

In [ ]:
# Achtung: diese Code tut vermutlich nicht, was Sie erwartet haben,
# weil im __init__()-Code der Basisklasse, diese (d.h. MyClass) direkt
# referenziert wird (und nicht MyOtherClass). Sie dazu Zeile 5 der Zelle
# in der MyClass definiert wird.
class MyOtherClass(MyClass):
    instance_counter = 0

a = MyOtherClass()
b = MyOtherClass()

Man kann das auch so schreiben, wodurch der Counter auch für Subklassen funktioniert:

In [ ]:
class MyClass:
    instance_counter = 0
    
    def __init__(self):
        # self.__class__ referenziert auf die eigene Klasse des Objekts
        self.__class__.instance_counter += 1
        print(f'Ich bin das {self.__class__.instance_counter}. Objekt')
        
a = MyClass()
b = MyClass()

In [ ]:
class MyOtherClass(MyClass):
    instance_counter = 0

a = MyOtherClass()
b = MyOtherClass()

<div class="alert alert-block alert-info">
<b>Übung Obj-3.1</b>
<p>
Schreiben Sie eine Klasse Student, die über eine Klassenvariable sicherstellt, dass keine Matrikelnummer mehr als einmal vorkommt.
</p>
</div>    

## Vertiefende Literatur zu diesem Abschnitt

Ich empfehle ausdrücklich, mindestens eine der folgenden Ressourcen zur Vertiefung zu lesen!

* Python Tutorial: Kapitel 9.1 - 9.6
	(http://docs.python.org/3/tutorial/classes.html)
* Klein, Kurs: 
	* Klassen (http://python-kurs.eu/python3_klassen.php)
	* Klassen- und Instanzattribute (http://python-kurs.eu/python3_klassen_instanzattribute.php)
	* Vererbung: (http://python-kurs.eu/python3_vererbung.php)
	* Mehrfachvererbung (http://python-kurs.eu/python3_mehrfachvererbung.php)

* Klein, Buch: Kapitel 19
* Kofler, Kapitel 11
* Weigend: Kapitel 10
* Briggs: Kapitel 9

* Downey: 
	* Kapitel 15: Classes and objects
	  (http://www.greenteapress.com/thinkpython/html/thinkpython016.html)
	* Kapitel 16: Classes and functions
	  (http://www.greenteapress.com/thinkpython/html/thinkpython017.html)
	* Kapitel 17: Classes and methods
	  (http://www.greenteapress.com/thinkpython/html/thinkpython018.html)
	* Kapitel 18: Inheritance
	  (http://www.greenteapress.com/thinkpython/html/thinkpython019.html)

## Lizenz

This notebook ist part of the course [Grundlagen der Programmierung](https://github.com/gvasold/gdp) held by [Gunter Vasold](https://online.uni-graz.at/kfu_online/wbForschungsportal.cbShowPortal?pPersonNr=51488) at Graz University 2017&thinsp;ff. 

<p>
    It is licensed under <a href="https://creativecommons.org/licenses/by-nc-sa/4.0">CC BY-NC-SA 4.0</a>
</p>

<table>
    <tr>
    <td>
        <img style="height:22px" 
             src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"/></li>
    </td>
    <td>
    <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
             src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1" /></li>
    </td>
</tr>
</table>